<a href="https://colab.research.google.com/github/armarmarm927/dl_lecture_competition_pub/blob/main/sparkproto16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine([-10,10],scale=(1,1.25)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.RandomErasing(p=0.5,scale=(0.02,0.33),ratio=(0.3,3.3),value=0,inplace=False)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [27]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

In [28]:
model_ft = models.resnet50(pretrained=True)
model_ft.fc = nn.Linear(model_ft.fc.in_features, 10)
net = model_ft.to(device)

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [30]:
num_epochs = 50
loss_list = []
acc_list = []
train_acc_list = []

for epoch in range(num_epochs):
    net.train()
    correct_train = 0
    total_train = 0
    epoch_loss = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    train_accuracy = 100. * correct_train / total_train
    train_acc_list.append(train_accuracy)
    print(f'Epoch {epoch+1}: Loss: {epoch_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%')
    loss_list.append(epoch_loss/len(train_loader))

    net.eval()
    correct = 0
    total = 0
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    test_accuracy = 100. * correct / total
    acc_list.append(test_accuracy)
    print(f'Test Accuracy: {test_accuracy:.2f}%')

    # 混同行列の出力
    cm = confusion_matrix(y_true, y_pred)
    print(f'Confusion Matrix (Epoch {epoch+1}):')
    print(cm)

    torch.save(net.state_dict(), '/content/drive/MyDrive/Weight_Epoch50.pth')
    scheduler.step()

Epoch 1: Loss: 1.1597, Train Accuracy: 60.15%
Test Accuracy: 72.86%
Confusion Matrix (Epoch 1):
[[845  19  23  14  13   6   6   2  60  12]
 [ 21 875   0   4   0  14   6   0  15  65]
 [104   7 536  28 110 126  66   9  10   4]
 [ 31  10  39 394  74 379  41  14   6  12]
 [ 41   3  25  20 809  57  22  20   3   0]
 [ 17   2  26  58  59 804   9  21   2   2]
 [  7   6  33  48  60  57 783   1   4   1]
 [ 61   2  15  19 140 134   9 606   2  12]
 [ 66  37   7  15   9   6   7   3 845   5]
 [ 64  77   3  10   1  15   5   4  32 789]]


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 2: Loss: 0.9996, Train Accuracy: 65.80%
Test Accuracy: 68.13%
Confusion Matrix (Epoch 2):
[[636  43  83  10   9   1   8   1 157  52]
 [ 17 906   4   4   0   6   5   0  22  36]
 [ 64  15 717  48  31  43  51   8  17   6]
 [ 41  17 116 585  22 112  59  15  27   6]
 [ 23   6 155 109 538  27  92  26  22   2]
 [ 22   9 126 189  26 585  17  13  12   1]
 [  9  21  80  81   7  10 783   2   7   0]
 [ 32   7  85 110  67  62  20 596   7  14]
 [ 39  72   6   7   5   4   4   1 847  15]
 [ 24 273   9  14   1  10  10   5  34 620]]


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 3: Loss: 0.9521, Train Accuracy: 67.66%
Test Accuracy: 76.70%
Confusion Matrix (Epoch 3):
[[771  32  42   6  11   2   4  10  96  26]
 [  6 885   5   1   0   4   3   1  30  65]
 [ 66   4 695  26  88  30  34  29  17  11]
 [ 17   5  98 561  56 150  42  32  24  15]
 [ 17   3  52  37 771  40  29  39  12   0]
 [  8   2  85 138  37 661  18  47   4   0]
 [  3   3  48  54  22  21 832   6   8   3]
 [ 21   5  32  36  41  39   2 807   4  13]
 [ 46  28  10   9   2   5   5   2 885   8]
 [ 26  90   6   9   1   6   2   8  50 802]]


...

Epoch 25: Loss: 0.3193, Train Accuracy: 88.89%
Test Accuracy: 86.98%
Confusion Matrix (Epoch 25):
[[915   8  17   7   6   0   4   2  29  12]
 [  6 944   0   4   0   4   1   0  14  27]
 [ 40   2 834  35  36  23  13  11   5   1]
 [ 15   3  39 745  27 121  24  18   5   3]
 [  6   1  38  29 859  19  16  26   6   0]
 [  6   0  20 127  20 790  11  21   4   1]
 [  5   2  29  38  12   7 900   2   2   3]
 [ 15   0  12  21  25  33   2 888   0   4]
 [ 33  14   4   6   3   3   0   0 930   7]
 [ 26  53   2   4   0   3   2   1  16 893]]


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 26: Loss: 0.3192, Train Accuracy: 88.85%
Test Accuracy: 87.11%
Confusion Matrix (Epoch 26):
[[906   8  20   9   8   0   4   3  30  12]
 [  6 942   1   4   0   3   1   1  14  28]
 [ 34   1 839  31  40  21  14  14   5   1]
 [ 14   3  38 739  34 120  25  18   5   4]
 [  5   1  36  29 865  19  16  24   5   0]
 [  5   0  21 117  21 802  10  20   3   1]
 [  5   2  26  40  13   7 901   2   1   3]
 [ 11   0  10  17  27  32   2 896   0   5]
 [ 37  13   5   7   3   4   0   0 923   8]
 [ 24  46   3   6   0   3   2   2  16 898]]


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
plt.plot(acc_list)
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.xticks(np.arange(0,50,10))
plt.ylabel('Accuracy')
plt.show()

In [ ]:
plt.plot(train_acc_list)
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.xticks(np.arange(0,50,10))
plt.ylabel('Accuracy')
plt.show()

In [ ]:
plt.plot(loss_list)
plt.title('Cross Entropy Loss')
plt.xlabel('Epoch')
plt.xticks(np.arange(0,50,10))
plt.ylabel('Accuracy')
plt.show()